In [ ]:
# Script para Análise Exploratória de Dados (EDA)
# Autor: Suelio Lima
# Email de contato: suelio@gmail.com
# GitHub: https://github.com/suelio
# Versão: 1.0

# Licença: MIT
# Permissão é concedida, gratuitamente, a qualquer pessoa que obtenha uma cópia
# deste software e arquivos de documentação associados (o "Software"), para lidar
# no Software sem restrição, incluindo, sem limitação, os direitos de usar, copiar,
# modificar, mesclar, publicar, distribuir, sublicenciar e/ou vender cópias do Software,
# e para permitir que as pessoas a quem o Software é fornecido o façam, sujeito às
# seguintes condições:
# O aviso de direitos autorais acima e este aviso de permissão devem ser incluídos em
# todas as cópias ou partes substanciais do Software.
# O SOFTWARE É FORNECIDO "COMO ESTÁ", SEM GARANTIA DE QUALQUER TIPO, EXPRESSA OU
# IMPLÍCITA, INCLUINDO, MAS NÃO SE LIMITANDO ÀS GARANTIAS DE COMERCIALIZAÇÃO,
# ADEQUAÇÃO A UM DETERMINADO FIM E NÃO VIOLAÇÃO. EM NENHUMA HIPÓTESE OS AUTORES OU
# TITULARES DOS DIREITOS AUTORAIS SERÃO RESPONSÁVEIS POR QUALQUER REIVINDICAÇÃO,
# DANOS OU OUTRA RESPONSABILIDAADE, SEJA EM UMA AÇÃO DE CONTRATO, ATO ILÍCITO OU OUTRA,
# DECORRENTE DE, FORA DE OU EM CONEXÃO COM O SOFTWARE OU O USO OU OUTRAS NEGOCIAÇÕES NO
# SOFTWARE.

# Instalação dos pacotes necessários (opcional, use apenas se precisar)
# !pip install pandas numpy seaborn matplotlib scipy scikit-learn

# Importação dos pacotes
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency, ttest_ind, f_oneway, pearsonr

# Funções do script
def load_data(file_path):
    """Carrega o dataset a partir de um arquivo CSV."""
    return pd.read_csv(file_path)

def basic_info(df):
    """Exibe informações básicas sobre o dataset."""
    print("Informações Básicas:")
    print(df.info())
    print("\nEstatísticas Descritivas:")
    print(df.describe(include='all'))
    print("\nValores Faltantes:")
    print(df.isnull().sum())

def detect_outliers(df, column):
    """Detecta outliers em uma coluna usando o método IQR."""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

def handle_missing_data(df):
    """Pergunta ao usuário se deseja tratar dados faltantes."""
    if df.isnull().sum().sum() > 0:
        print("\nValores Faltantes Encontrados:")
        print(df.isnull().sum())
        choice = input("Deseja tratar os dados faltantes? (s/n): ").strip().lower()
        if choice == 's':
            for col in df.columns:
                if df[col].isnull().sum() > 0:
                    if df[col].dtype == 'object':
                        df[col] = df[col].fillna(df[col].mode()[0])  # Preenche com a moda para colunas categóricas
                    else:
                        df[col] = df[col].fillna(df[col].median())  # Preenche com a mediana para colunas numéricas
            print("\nDados Faltantes Após Tratamento:")
            print(df.isnull().sum())
        else:
            print("Dados faltantes não foram tratados.")
    else:
        print("\nNenhum valor faltante encontrado.")

def handle_outliers(df):
    """Pergunta ao usuário o que fazer com os outliers."""
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        outliers = detect_outliers(df, col)
        if not outliers.empty:
            print(f"\nOutliers encontrados na coluna '{col}':")
            print(outliers)
            choice = input(f"Deseja remover os outliers da coluna '{col}'? (s/n): ").strip().lower()
            if choice == 's':
                Q1 = df[col].quantile(0.25)
                Q3 = df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 1.5 * IQR
                upper_bound = Q3 + 1.5 * IQR
                df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
                print(f"Outliers da coluna '{col}' removidos.")
            else:
                print(f"Outliers da coluna '{col}' mantidos.")
    return df

def suggest_analysis(df):
    """Sugere análises com base nos tipos de dados."""
    suggestions = []
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object']).columns

    if len(numeric_cols) > 1:
        suggestions.append("1. Teste de correlação de Pearson")
    if len(categorical_cols) > 1:
        suggestions.append("2. Teste de qui-quadrado")
    if len(numeric_cols) >= 1 and len(categorical_cols) >= 1:
        suggestions.append("3. Teste t de Student")
        suggestions.append("4. ANOVA")
    suggestions.append("5. Rodar todas as análises")
    suggestions.append("6. Nenhuma (encerrar o script)")

    print("\nRecomendações de Análises:")
    for suggestion in suggestions:
        print(suggestion)

    return suggestions, numeric_cols, categorical_cols

def check_p_value(p_val, alpha=0.05):
    """Verifica se o p-valor é significativo."""
    if np.isnan(p_val):
        print("Erro: Não foi possível calcular o p-valor. Verifique os dados.")
        return False
    elif p_val < alpha:
        print(f"O p-valor é {p_val:.4f}, que é menor que o nível de significância (α = {alpha}).")
        print("Interpretação: Há evidências fortes de que há uma diferença ou associação significativa.")
        return True
    else:
        print(f"O p-valor é {p_val:.4f}, que é maior ou igual ao nível de significância (α = {alpha}).")
        print("Interpretação: Não há evidências suficientes para concluir que há uma diferença ou associação significativa.")
        return False

def explain_chi2(chi2_stat, p_val):
    """Explica o resultado do teste de qui-quadrado."""
    print("\nInterpretação do Teste de Qui-Quadrado:")
    print("O teste de qui-quadrado avalia se há uma associação significativa entre duas variáveis categóricas.")
    print(f"Estatística qui-quadrado: {chi2_stat:.4f}")
    if check_p_value(p_val):
        print("Conclusão: As variáveis estão associadas de forma significativa.")
    else:
        print("Conclusão: As variáveis não estão associadas de forma significativa.")

def perform_default_analysis(df, numeric_cols):
    """Realiza análises padrão (medidas de posição, dispersão, resumo de cinco números, gráficos)."""
    print("\n=== Análises Padrão ===")

    # Medidas de posição e dispersão
    for col in numeric_cols:
        print(f"\nAnálise da coluna '{col}':")
        print(f"Média: {df[col].mean():.4f}")
        print(f"Mediana: {df[col].median():.4f}")
        print(f"Variância: {df[col].var():.4f}")
        print(f"Desvio Padrão: {df[col].std():.4f}")

    # Resumo de cinco números
    print("\nResumo de Cinco Números:")
    for col in numeric_cols:
        print(f"\nColuna '{col}':")
        print(f"Mínimo: {df[col].min():.4f}")
        print(f"Q1 (25%): {df[col].quantile(0.25):.4f}")
        print(f"Mediana (50%): {df[col].median():.4f}")
        print(f"Q3 (75%): {df[col].quantile(0.75):.4f}")
        print(f"Máximo: {df[col].max():.4f}")

    # Histogramas e boxplots
    print("\nGráficos:")
    for col in numeric_cols:
        plt.figure(figsize=(12, 4))

        # Histograma
        plt.subplot(1, 2, 1)
        sns.histplot(df[col], kde=True)
        plt.title(f"Histograma de {col}")

        # Boxplot
        plt.subplot(1, 2, 2)
        sns.boxplot(y=df[col])
        plt.title(f"Boxplot de {col}")

        plt.show()

def perform_analysis(df, choice, numeric_cols, categorical_cols):
    """Executa a análise escolhida pelo usuário."""
    while True:
        if choice == 1:
            # Correlação de Pearson
            if len(numeric_cols) > 1:
                corr_matrix = df[numeric_cols].corr()
                sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
                plt.title("Matriz de Correlação de Pearson")
                plt.show()
                break
            else:
                print("Não há colunas numéricas suficientes para calcular a correlação de Pearson.")
                break

        elif choice == 2:
            # Teste de qui-quadrado
            if len(categorical_cols) >= 2:
                print("\nColunas categóricas disponíveis:")
                for i, col in enumerate(categorical_cols):
                    print(f"{i + 1}. {col}")
                try:
                    col1 = int(input("Escolha o número da primeira coluna: ")) - 1
                    col2 = int(input("Escolha o número da segunda coluna: ")) - 1
                    if col1 < 0 or col1 >= len(categorical_cols) or col2 < 0 or col2 >= len(categorical_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    cross_tab = pd.crosstab(df[categorical_cols[col1]], df[categorical_cols[col2]])
                    chi2_stat, p_val, dof, expected = chi2_contingency(cross_tab)
                    if np.isnan(p_val):
                        print("Erro: Não foi possível calcular o p-valor. As colunas escolhidas não são adequadas.")
                        print("Por favor, escolha outras colunas.")
                        continue
                    print(f"\nEstatística qui-quadrado: {chi2_stat:.4f}, p-valor: {p_val:.4f}")
                    explain_chi2(chi2_stat, p_val)
                    break
                except ValueError:
                    print("Erro: Entrada inválida. Digite apenas números.")
                    continue
            else:
                print("Não há colunas categóricas suficientes para realizar o teste de qui-quadrado.")
                break

        elif choice == 3:
            # Teste t de Student
            if len(numeric_cols) >= 1 and len(categorical_cols) >= 1:
                print("\nColunas numéricas disponíveis:")
                for i, col in enumerate(numeric_cols):
                    print(f"{i + 1}. {col}")
                try:
                    num_col = int(input("Escolha o número da coluna numérica: ")) - 1
                    if num_col < 0 or num_col >= len(numeric_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    print("Colunas categóricas disponíveis:")
                    for i, col in enumerate(categorical_cols):
                        print(f"{i + 1}. {col}")
                    cat_col = int(input("Escolha o número da coluna categórica: ")) - 1
                    if cat_col < 0 or cat_col >= len(categorical_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    group1 = df[df[categorical_cols[cat_col]] == df[categorical_cols[cat_col]].unique()[0]][numeric_cols[num_col]]
                    group2 = df[df[categorical_cols[cat_col]] == df[categorical_cols[cat_col]].unique()[1]][numeric_cols[num_col]]
                    t_stat, p_val = ttest_ind(group1, group2)
                    if np.isnan(p_val):
                        print("Erro: Não foi possível calcular o p-valor. As colunas escolhidas não são adequadas.")
                        print("Por favor, escolha outras colunas.")
                        continue
                    print(f"\nEstatística t: {t_stat:.4f}, p-valor: {p_val:.4f}")
                    check_p_value(p_val)
                    break
                except ValueError:
                    print("Erro: Entrada inválida. Digite apenas números.")
                    continue
            else:
                print("Não há colunas numéricas e categóricas suficientes para realizar o teste t de Student.")
                break

        elif choice == 4:
            # ANOVA
            if len(numeric_cols) >= 1 and len(categorical_cols) >= 1:
                print("\nColunas numéricas disponíveis:")
                for i, col in enumerate(numeric_cols):
                    print(f"{i + 1}. {col}")
                try:
                    num_col = int(input("Escolha o número da coluna numérica: ")) - 1
                    if num_col < 0 or num_col >= len(numeric_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    print("Colunas categóricas disponíveis:")
                    for i, col in enumerate(categorical_cols):
                        print(f"{i + 1}. {col}")
                    cat_col = int(input("Escolha o número da coluna categórica: ")) - 1
                    if cat_col < 0 or cat_col >= len(categorical_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    groups = [df[df[categorical_cols[cat_col]] == category][numeric_cols[num_col]] for category in df[categorical_cols[cat_col]].unique()]
                    f_stat, p_val = f_oneway(*groups)
                    if np.isnan(p_val):
                        print("Erro: Não foi possível calcular o p-valor. As colunas escolhidas não são adequadas.")
                        print("Por favor, escolha outras colunas.")
                        continue
                    print(f"\nEstatística F: {f_stat:.4f}, p-valor: {p_val:.4f}")
                    if not check_p_value(p_val):
                        print("Atenção: A análise pode não ser válida devido a dados insuficientes ou problemas nos dados.")
                    break
                except ValueError:
                    print("Erro: Entrada inválida. Digite apenas números.")
                    continue
            else:
                print("Não há colunas numéricas e categóricas suficientes para realizar a ANOVA.")
                break

        elif choice == 5:
            # Rodar todas as análises
            print("\nRodando todas as análises disponíveis...")
            if len(numeric_cols) > 1:
                print("\n1. Teste de correlação de Pearson:")
                corr_matrix = df[numeric_cols].corr()
                sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
                plt.title("Matriz de Correlação de Pearson")
                plt.show()

            if len(categorical_cols) >= 2:
                print("\n2. Teste de qui-quadrado:")
                print("Colunas categóricas disponíveis:")
                for i, col in enumerate(categorical_cols):
                    print(f"{i + 1}. {col}")
                try:
                    col1 = int(input("Escolha o número da primeira coluna: ")) - 1
                    col2 = int(input("Escolha o número da segunda coluna: ")) - 1
                    if col1 < 0 or col1 >= len(categorical_cols) or col2 < 0 or col2 >= len(categorical_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    cross_tab = pd.crosstab(df[categorical_cols[col1]], df[categorical_cols[col2]])
                    chi2_stat, p_val, dof, expected = chi2_contingency(cross_tab)
                    if np.isnan(p_val):
                        print("Erro: Não foi possível calcular o p-valor. As colunas escolhidas não são adequadas.")
                        print("Por favor, escolha outras colunas.")
                        continue
                    print(f"\nEstatística qui-quadrado: {chi2_stat:.4f}, p-valor: {p_val:.4f}")
                    explain_chi2(chi2_stat, p_val)
                except ValueError:
                    print("Erro: Entrada inválida. Digite apenas números.")
                    continue

            if len(numeric_cols) >= 1 and len(categorical_cols) >= 1:
                print("\n3. Teste t de Student:")
                print("Colunas numéricas disponíveis:")
                for i, col in enumerate(numeric_cols):
                    print(f"{i + 1}. {col}")
                try:
                    num_col = int(input("Escolha o número da coluna numérica: ")) - 1
                    if num_col < 0 or num_col >= len(numeric_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    print("Colunas categóricas disponíveis:")
                    for i, col in enumerate(categorical_cols):
                        print(f"{i + 1}. {col}")
                    cat_col = int(input("Escolha o número da coluna categórica: ")) - 1
                    if cat_col < 0 or cat_col >= len(categorical_cols):
                        print("Erro: Número de coluna inválido. Escolha números dentro do intervalo disponível.")
                        continue
                    group1 = df[df[categorical_cols[cat_col]] == df[categorical_cols[cat_col]].unique()[0]][numeric_cols[num_col]]
                    group2 = df[df[categorical_cols[cat_col]] == df[categorical_cols[cat_col]].unique()[1]][numeric_cols[num_col]]
                    t_stat, p_val = ttest_ind(group1, group2)
                    if np.isnan(p_val):
                        print("Erro: Não foi possível calcular o p-valor. As colunas escolhidas não são adequadas.")
                        print("Por favor, escolha outras colunas.")
                        continue
                    print(f"\nEstatística t: {t_stat:.4f}, p-valor: {p_val:.4f}")
                    check_p_value(p_val)
                except ValueError:
                    print("Erro: Entrada inválida. Digite apenas números.")
                    continue

                print("\n4. ANOVA:")
                groups = [df[df[categorical_cols[cat_col]] == category][numeric_cols[num_col]] for category in df[categorical_cols[cat_col]].unique()]
                f_stat, p_val = f_oneway(*groups)
                if np.isnan(p_val):
                    print("Erro: Não foi possível calcular o p-valor. As colunas escolhidas não são adequadas.")
                    print("Por favor, escolha outras colunas.")
                    continue
                print(f"\nEstatística F: {f_stat:.4f}, p-valor: {p_val:.4f}")
                if not check_p_value(p_val):
                    print("Atenção: A análise pode não ser válida devido a dados insuficientes ou problemas nos dados.")
            break

        elif choice == 6:
            # Encerrar o script
            print("Nenhuma análise selecionada. Encerrando o script.")
            return

        else:
            print("Opção inválida. Escolha um número válido.")
            break

def main():
    # Solicita o caminho do arquivo ao usuário
    file_path = input("Digite o caminho/nome do arquivo CSV que deseja analisar: ").strip()

    # Carrega os dados
    try:
        df = load_data(file_path)
        print(f"Arquivo '{file_path}' carregado com sucesso!")
    except FileNotFoundError:
        print(f"Erro: Arquivo '{file_path}' não encontrado. Verifique o nome e o caminho do arquivo.")
        return

    # Exibe informações básicas
    basic_info(df)

    # Pergunta se deseja tratar dados faltantes
    handle_missing_data(df)

    # Pergunta o que fazer com os outliers
    df = handle_outliers(df)

    # Realiza análises padrão
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    perform_default_analysis(df, numeric_cols)

    # Sugere e realiza análises
    while True:
        suggestions, numeric_cols, categorical_cols = suggest_analysis(df)
        if suggestions:
            try:
                choice = int(input("\nEscolha o número da análise que deseja realizar: "))
                if choice == 6:
                    print("Encerrando o script.")
                    break
                perform_analysis(df, choice, numeric_cols, categorical_cols)

                # Pergunta se deseja realizar outra análise
                repeat = input("\nDeseja realizar outra análise? (s/n): ").strip().lower()
                if repeat != 's':
                    print("Encerrando o script.")
                    break
            except ValueError:
                print("Erro: Entrada inválida. Digite apenas números.")
        else:
            print("Nenhuma análise recomendada com base nos dados disponíveis.")
            break

if __name__ == "__main__":
    main()